# DOSTA Calibration Parser

This script and notebook gives the code for loading the CTD calibration coefficients into a properly named calibration csv, as well as an example of how to use it. The calibration coefficients can be parsed from either the vendor calibration (.cal) file, the vendor xmlcon file, or from the capture file (either .cap, .log, or .txt) from the QCT check-in.


In [1]:
from utils import *

In [2]:
import csv
import datetime
import os
import shutil
import sys
import time
import re
import xml.etree.ElementTree as et
import pandas as pd
from zipfile import ZipFile

========================================================================================================================
### Define functions

In [3]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [4]:
def get_qct_files(df, qct_directory):
    qct_dict = {}
    uids = list(set(df['UID']))
    for uid in uids:
        df['UID_match'] = df['UID'].apply(lambda x: True if uid in x else False)
        qct_series = df[df['UID_match'] == True]['QCT Testing']
        qct_series = list(str(qct_series.iloc[0]).split('\n'))
        qct_dict.update({uid:qct_series})
    return qct_dict

In [5]:
# Now I need to load the all of the csv files based on their UID
def load_csv_info(csv_dict,filepath):
    """
    Loads the calibration coefficient information contained in asset management
    
    Args:
        csv_dict - a dictionary which associates an instrument UID to the
            calibration csv files in asset management
        filepath - the path to the directory containing the calibration csv files
    Returns:
        csv_cals - a dictionary which associates an instrument UID to a pandas
            dataframe which contains the calibration coefficients. The dataframes
            are indexed by the date of calibration
    """
    
    # Load the calibration data into pandas dataframes, which are then placed into
    # a dictionary by the UID
    csv_cals = {}
    for uid in csv_dict:
        cals = pd.DataFrame()
        for file in csv_dict[uid]:
            data = pd.read_csv(filepath+file)
            date = file.split('__')[1].split('.')[0]
            data['CAL DATE'] = pd.to_datetime(date)
            cals = cals.append(data)
        csv_cals.update({uid:cals})
        
    # Pivot the dataframe to be sorted based on calibration date
    for uid in csv_cals:
        csv_cals[uid] = csv_cals[uid].pivot(index=csv_cals[uid]['CAL DATE'], columns='name')['value']
        
    return csv_cals

In [6]:
def splitDataFrameList(df,target_column):
    ''' 
    df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
            
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df

========================================================================================================================
### Directories
**Define the main directories where important information is stored.**

In [29]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/PRESF/PRESF_Results'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/PRESF/PRESF_Cal'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/PRESFB'

In [18]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [37]:
PRESF = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='PRESF',whoi=True)
PRESF

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
1057,PRESF,B,26-1392,116898,A01141,CGINS-PRESFB-01392,26Plus,3305-00012-00002,7.2,SeaBird,...,3305-00105-00018\n3305-00105-00045\n3305-00105...,NaN,NaN,3305-00900-00178\n3305-00900-00361,WHOI_11-09-2011-PRESF-1003,10/30/2014\n4/3/2015,CP01CNSM-00005\nCP8 Spare\nCP01CNSM-00008\nCP0...,NaN,MFN,NaN
1058,PRESF,C,26-1400,116909,A01187,CGINS-PRESFC-01400,26Plus,3305-00012-00003,7.2,SeaBird,...,3305-00105-00019\n3305-00105-00028\n3305-00108...,NaN,NaN,3305-00900-00085,WHOI_11-09-2011-PRESF-1003,2014-12-12 00:00:00,CP04OSSM-00002\nCP04OSSM-00005\nCP04OSSM-00008,Pioneer 12 spare,NaN,NaN
1059,PRESF,C,26-1401,116910,A01188,CGINS-PRESFC-01401,26Plus,3305-00012-00003,7.2,SeaBird,...,3305-00105-00020\n3305-00105-00037\n3305-00105...,NaN,NaN,3305-00900-00135\n3305-00900-00309,WHOI_11-09-2011-PRESF-1003,2014-12-12 00:00:00,CP4 Spare\nCP04OSSM-00003\nCP04OSSM-00006\nP10...,CP04OSSM-00010,MFN,NaN
1061,PRESF,B,26P64515-1329,114940,A00051,CGINS-PRESFB-01329,26Plus,3305-00012-00002,6.1e,SeaBird,...,3305-00105-00002\n3305-00105-00027\n3305-00105...,NaN,NaN,3305-00900-00085\n3305-00900-00178\n3305-00900...,WH-DO-1,2012-04-09 00:00:00,CP1 Spare\nCP3a Spare\nCP03ISSM-00002\nCP03ISS...,NaN,MFN,NaN
1062,PRESF,B,26P71826-1352,115275,A00165,CGINS-PRESFB-01352,26Plus,3305-00012-00002,7.2,SeaBird,...,3305-00105-00003\n3305-00105-00038\n3305-00105...,NaN,NaN,3305-00900-00013\n3305-00900-00135\n3305-00900...,WHOI_Contract_11-09-2011-PRESF-1001,2813-01-01 00:00:00,CP01CNSM-00001\nCP4 Spare\nCP03ISSM-00003\nCP0...,CP01CNSM-00011,MFN,NaN
1063,PRESF,B,26P76848-1386,116453,A00813,CGINS-PRESFB-01386,26Plus,3305-00012-00002,7.2,SeaBird,...,3305-00105-00010\n3305-00105-00025\n3305-00105...,NaN,NaN,3305-00900-00085\n3305-00900-00178\n3305-00900...,WHOI_Contract_11-09-2011-PRESF-1002,2014-04-11 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP04OSSM-00004...,CP03ISSM-00010,MFN,NaN
1064,PRESF,B,26P76848-1387,116337,A00761,CGINS-PRESFB-01387,26Plus,3305-00012-00002,7.2,SeaBird,...,3305-00105-00011\n3305-00105-00026\n3305-00105...,NaN,NaN,3305-00900-00085,WHOI_Contract_11-09-2011-PRESF-1002,2014-04-11 00:00:00,CP03ISSM-00001\nCP01CNSM-00006\nCP03ISSM-00008,Pioneer 12 spare,NaN,NaN
1065,PRESF,C,26P76848-1388,116338,A00762,CGINS-PRESFC-01388,26Plus,3305-00012-00003,7.2,SeaBird,...,3305-00105-00012\n3305-00105-00024\n3305-00105...,NaN,NaN,3305-00900-00035\n3305-00900-00364,WHOI_Contract_11-09-2011-PRESF-1002,2014-04-11 00:00:00,CP04OSSM-00001\nCP8 Spare\nCP04OSSM-00007\nCP0...,NaN,MFN,NaN


**Identify the QCT Testing documents associated with each individual instrument (the UID)**

In [34]:
qct_dict = get_qct_files(PRESF, qct_directory)
qct_dict

{'CGINS-PRESFB-01387': ['3305-00105-00011',
  '3305-00105-00026',
  '3305-00105-00046'],
 'CGINS-PRESFB-01329': ['3305-00105-00002',
  '3305-00105-00027',
  '3305-00105-00043',
  '3305-00105-00054'],
 'CGINS-PRESFC-01401': ['3305-00105-00020',
  '3305-00105-00037',
  '3305-00105-00055'],
 'CGINS-PRESFB-01392': ['3305-00105-00018',
  '3305-00105-00045',
  '3305-00105-00066'],
 'CGINS-PRESFC-01400': ['3305-00105-00019',
  '3305-00105-00028',
  '3305-00108-00048'],
 'CGINS-PRESFB-01386': ['3305-00105-00010',
  '3305-00105-00025',
  '3305-00105-00044',
  '3305-00105-00056'],
 'CGINS-PRESFA-01328': ['3305-00105-00001'],
 'CGINS-PRESFC-01388': ['3305-00105-00012',
  '3305-00105-00024',
  '3305-00105-00067'],
 'CGINS-PRESFB-01352': ['3305-00105-00003',
  '3305-00105-00038',
  '3305-00105-00047',
  '3305-00105-00065']}

**Identify the calibration csvs stored in asset management which correspond to a particular instrument.**

In [31]:
csv_dict = load_asset_management(PRESF, asset_management_directory)
csv_dict

{'CGINS-PRESFB-01329': ['CGINS-PRESFB-01329__20150509.csv',
  'CGINS-PRESFB-01329__20170323.csv',
  'CGINS-PRESFB-01329__20171217.csv',
  'CGINS-PRESFB-01329__20160528.csv'],
 'CGINS-PRESFB-01352': ['CGINS-PRESFB-01352__20180620.csv',
  'CGINS-PRESFB-01352__20131121.csv',
  'CGINS-PRESFB-01352__20170811.csv',
  'CGINS-PRESFB-01352__20151021.csv',
  'CGINS-PRESFB-01352__20161011.csv'],
 'CGINS-PRESFB-01386': ['CGINS-PRESFB-01386__20160527.csv',
  'CGINS-PRESFB-01386__20170323.csv',
  'CGINS-PRESFB-01386__20141213.csv',
  'CGINS-PRESFB-01386__20171217.csv',
  'CGINS-PRESFB-01386__20150507.csv'],
 'CGINS-PRESFB-01387': ['CGINS-PRESFB-01387__20170725.csv',
  'CGINS-PRESFB-01387__20141214.csv'],
 'CGINS-PRESFB-01392': ['CGINS-PRESFB-01392__20160513.csv',
  'CGINS-PRESFB-01392__20180604.csv',
  'CGINS-PRESFB-01392__20170104.csv']}

In [35]:
for uid in csv_dict.keys():
    for file in sorted(csv_dict[uid]):
        print(file)

CGINS-PRESFB-01329__20150509.csv
CGINS-PRESFB-01329__20160528.csv
CGINS-PRESFB-01329__20170323.csv
CGINS-PRESFB-01329__20171217.csv
CGINS-PRESFB-01352__20131121.csv
CGINS-PRESFB-01352__20151021.csv
CGINS-PRESFB-01352__20161011.csv
CGINS-PRESFB-01352__20170811.csv
CGINS-PRESFB-01352__20180620.csv
CGINS-PRESFB-01386__20141213.csv
CGINS-PRESFB-01386__20150507.csv
CGINS-PRESFB-01386__20160527.csv
CGINS-PRESFB-01386__20170323.csv
CGINS-PRESFB-01386__20171217.csv
CGINS-PRESFB-01387__20141214.csv
CGINS-PRESFB-01387__20170725.csv
CGINS-PRESFB-01392__20160513.csv
CGINS-PRESFB-01392__20170104.csv
CGINS-PRESFB-01392__20180604.csv


In [13]:
uids = sorted(list(csv_dict.keys()))

In [14]:
serial_nums = get_serial_nums(DOSTA, uids)


In [15]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

{'CGINS-DOSTAD-00126': ['DOSTA_Optode-4831_SN_126_Calibration_Certificate_2016-03-11.pdf',
  'DOSTA_Optode-4831_SN_126_Calibration_Certificate_2017-08-19.pdf',
  'DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2013-02-27.pdf',
  'DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2018-03-09.pdf'],
 'CGINS-DOSTAD-00127': ['DOSTA_Optode-4831_SN_127_Calibration_Certificate_2018-09-20.pdf',
  'DOSTA_Optode-4831_SN_127_Multipoint_Calibration_2013-03-04.pdf',
  'DOSTA_Optode-4831_SN_127_Multipoint_Calibration_2016-02-01.pdf'],
 'CGINS-DOSTAD-00128': ['DOSTA_Optode-4831_SN_128_Calibration_Certificate_2017-09-11.pdf',
  'DOSTA_Optode-4831_SN_128_Multipoint_Calibration_2013-03-04.pdf',
  'DOSTA_Optode-4831_SN_128_Multipoint_Calibration_2016-02-01.pdf'],
 'CGINS-DOSTAD-00129': ['DOSTA_Optode-4831_SN_129_Calibration_Certificate_2018-09-20.pdf',
  'DOSTA_Optode-4831_SN_129_Multipoint_Calibration_2013-02-27.pdf',
  'DOSTA_Optode-4831_SN_129_Multipoint_Calibration_2016-02-01.pdf'],
 'CGINS-DOSTAD-0013

========================================================================================================================
**Now, need to get all the files for a particular CTDMO UID:**

In [61]:
uid = sorted(uids)[0]
uid

'CGINS-DOSTAD-00126'

In [62]:
cal_files = sorted(cal_dict[uid])
for file in cal_files:
    print(file)

DOSTA_Optode-4831_SN_126_Calibration_Certificate_2016-03-11.pdf
DOSTA_Optode-4831_SN_126_Calibration_Certificate_2017-08-19.pdf
DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2013-02-27.pdf
DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2018-03-09.pdf


In [63]:
csv_files = sorted(csv_dict[uid])
for file in csv_files:
    print(file)

CGINS-DOSTAD-00126__20140916.csv
CGINS-DOSTAD-00126__20161011.csv
CGINS-DOSTAD-00126__20170819.csv
CGINS-DOSTAD-00126__20180309.csv


In [64]:
qct_files = sorted(qct_dict[uid])
for file in qct_files:
    print(file)

3305-00115-00128
3305-00115-00234
3305-00115-00259


In [65]:
csv_path = []
for cf in csv_files:
    path = generate_file_path(asset_management_directory, cf)
    csv_path.append(path)
csv_path

['/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD/CGINS-DOSTAD-00126__20140916.csv',
 '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD/CGINS-DOSTAD-00126__20161011.csv',
 '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD/CGINS-DOSTAD-00126__20170819.csv',
 '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/DOSTAD/CGINS-DOSTAD-00126__20180309.csv']

In [66]:
cal_path = []
for cf in cal_files:
    path = generate_file_path(cal_directory, cf)
    cal_path.append(path)
cal_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Cal/DOSTA_Optode-4831_SN_126_Calibration_Certificate_2016-03-11.pdf',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Cal/DOSTA_Optode-4831_SN_126_Calibration_Certificate_2017-08-19.pdf',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Cal/DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2013-02-27.pdf',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Cal/DOSTA_Optode-4831_SN_126_Multipoint_Calibration_2018-03-09.pdf']

In [67]:
qct_path = []
for qf in qct_files:
    path = generate_file_path(qct_directory, qf)
    qct_path.append(path)
qct_path

['/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Results/3305-00115-00128-A.log',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Results/3305-00115-00234-A.log',
 '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/DOSTA/DOSTA_Results/3305-00115-00259-A.txt']

### What does the DOSTA Calibration file format look like?

1. The calibration files are all pdf files sent by SeaBird. They are most likely NOT machine readable, which presents a challenge in automating this approach.

2. Conclusion: I won't try to read the pdfs. Instead, I'll focus on writing the csv cal sheet from the qct checkin info.

In [69]:
import textract

In [77]:
ptext = textract.process(cal_path[1], method='tesseract', encoding='utf-8')
print(ptext.decode('utf-8'))

 

VAANDERAA CALIBRATION CERTIFICATE

Form No 830, Juli 2012

 

a xylem brand
Certificate no: 4831_126_00133171 Product: 4831 Serial no: 126
Foil batch no: 1517M Calibration date: 19.08.2017 Page 1 of 2
Ps Porson omen] oun [eran enim | reams |
0 30.014 0.49 -220.380 60.41
1 19.698 0.32 107.735 61.22
2 9,716 0.58 435.515 61.93
3 0.577 157 721,790 62.54
4 0.649 22.56 719,645 59.71
5 0.719 44.81 717.520 57.04
6 0.757 65.11 716,415 54.85
7 0.733 105,73 717.135 51.07
8 0.724 145.47 717.410 47.98
9 0.697 212.40 718.210 43,80
10 0.661 312.96 719,300 39.09
Ml 0.641 417.89 719.875 35.46
12 0.659 518.93 719,340 32.81
13 10,137 13.11 421.875 59.63
14 10.049 34.26 424,720 55.93
15 9.962 50.31 427.585 53.53
16 9.902 81.33 429,540 49.56
7 9,854 113.61 431,080 46.20
18 9.789 166.71 433.195 41.85
19 9,722 245.91 435.340 IAL
20 9.698 328.21 436.125 33.58
a 9.716 410.07 435,555 30,94
22 19.827 9.57 103.500 58.84
23 19,753 24.38 105,940 55,17
24 15,699 36.87 107.685 52.52
25 19.671 62.73 108.620 47,99
2

In [ ]:
ptext = ptext.replace(b'\xe2\x80\x94',b'-')
ptext = ptext.decode('utf-8')

In [58]:
pd.read_csv(csv_path[0])

,serial,name,value,notes
0,126,CC_conc_coef,"[0.0, 1.0]",default
1,126,CC_csv,"[0.002723472, 0.0001146171, 2.127325e-06, 233....",NaN


### What do the DOSTA QCT file format look like?

In [ ]:
#!/usr/bin/env python
import csv
import datetime
import os
import shutil
import sys
import time
import re
import xml.etree.ElementTree as et
import pandas as pd
from zipfile import ZipFile


class DOSTACalibration():
    # Class that stores calibration values for DOSTA's.

    def __init__(self, uid, calibration_date):
        self.serial = ''
        self.uid = uid
        self.date = pd.to_datetime(calibration_date).strftime('%Y%m%d')
        self.coefficients = {'CC_conc_coef':None,
                            'CC_csv':None}
        self.notes = {'CC_conc_coef':None,
                      'CC_csv':None}
                
    @property
    def uid(self):
        return self._uid
        
    @uid.setter
    def uid(self, d):
        r = re.compile('.{5}-.{6}-.{5}')
        if r.match(d) is not None:
            self.serial = d.split('-')[2]
            self._uid = d
        else:
            raise Exception(f"The instrument uid {d} is not a valid uid. Please check.")
            
            
    def generate_file_path(self,dirpath,filename,ext=['.cap','.txt','.log'],exclude=['_V','_Data_Workshop']):
        """
        Function which searches for the location of the given file and returns
        the full path to the file.
        
        Args:
            dirpath - parent directory path under which to search
            filename - the name of the file to search for
            ext - file endings to search for
            exclude - optional list which allows for excluding certain
                directories from the search
        Returns:
            fpath - the file path to the filename from the current
                working directory.
        """
        # Check if the input file name has an extension already
        # If it does, parse it for input into the search algo
        if '.' in filename:
            check = filename.split('.')
            filename = check[0]
            ext = ['.'+check[1]]
        
        for root, dirs, files in os.walk(dirpath):
            dirs[:] = [d for d in dirs if d not in exclude]
            for fname in files:
                if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                    fpath = os.path.join(root, fname)
                    return fpath
                
    def load_qct(self, filepath):
        """
        Function which parses the output from the QCT check-in and loads them into
        the DOSTA object.
        
        Args:
            filepath - the full directory path and filename 
        Raises:
            ValueError - checks if the serial number parsed from the UID matches the
                the serial number stored in the file.
        Returns:
            self.coefficients - populated coefficients dictionary
            self.date - the calibration dates associated with the calibration values
            self.type - the type (i.e. 16+/37-IM) of the CTD
            self.serial - populates the 5-digit serial number of the instrument 
        """
        
        data = {}
        with open(filepath, errors='ignore') as file:
            reader = csv.reader(file, delimiter='\t')
            for row in reader:
                data.update({reader.line_num:row})
                
        for key,info in data.items():
            # Find the serial number from the QCT check-in and compare to UID
            if 'serial number' in [x.lower() for x in info]:
                serial_num = info[-1].zfill(5)
                if self.serial != serial_num:
                    raise ValueError(f'Serial number {serial_num.zfill(5)} from the QCT file does not match {self.serial} from the UID.')
                else:
                    pass
                
            # Find the svu foil coefficients
            if 'svufoilcoef' in [x.lower() for x in info]:
                self.coefficients['CC_csv'] = [float(n) for n in info[3:]]
            
            # Find the concentration coefficients
            if 'conccoef' in [x.lower() for x in info]:
                self.coefficients['CC_conc_coef'] = [float(n) for n in info[3:]]
                
    def add_notes(self, notes):
        """
        This function adds notes to the calibration csv based on the 
        calibration coefficients.
        
        Args:
            notes - a dictionary with keys of the calibration coefficients
                which correspond to an entry of desired notes about the 
                corresponding coefficients
        Returns:
            self.notes - a dictionary with the entered notes.
        """
        keys = notes.keys()
        for key in keys:
            self.notes[key] = notes[key]


    def write_csv(self, outpath):
        """
        This function writes the correctly named csv file for the ctd to the
        specified directory.
        
        Args:
            outpath - directory path of where to write the csv file
        Raises:
            ValueError - raised if the CTD object's coefficient dictionary 
                has not been populated
        Returns:
            self.to_csv - a csv of the calibration coefficients which is 
                written to the specified directory from the outpath.
        """
        
        # Run a check that the coefficients have actually been loaded
        for key in self.coefficients.keys():
            if self.coefficients[key] == None:
                raise ValueError(f'No coefficients for {key} have been loaded.')
            
        # Create a dataframe to write to the csv
        data = {'serial':[self.serial]*len(self.coefficients),
               'name':list(self.coefficients.keys()),
               'value':list(self.coefficients.values()),
               'notes':list(self.notes.values()) }
        df = pd.DataFrame().from_dict(data)
        
        # Generate the csv name
        csv_name = self.uid + '__' + self.date + '.csv'
        
        # Now write to 
        check = input(f"Write {csv_name} to {outpath}? [y/n]: ")
        if check.lower().strip() == 'y':
            df.to_csv(outpath+'/'+csv_name, index=False)

In [ ]:
DOSTA = DOSTACalibration(uid='CGINS-DOSTAD-00126',calibration_date='March 22, 2019')

In [ ]:
DOSTA.date

In [ ]:
filepath = DOSTA.generate_file_path(qct_directory,'3305-00115-00128')
filepath

In [ ]:
DOSTA.load_qct(filepath)

In [ ]:
DOSTA.coefficients

In [ ]:
DOSTA.serial

In [ ]:
DOSTA.add_notes({'CC_conc_coef':'[intercept,slope]'})

In [ ]:
outdir = '/'.join((os.getcwd(),'temp'))

In [ ]:
DOSTA.write_csv(outdir)

In [ ]:
out = generate_file_path(asset_management_directory, csv_files[0].split('.')[0],ext=['.csv'])
print(out)

In [ ]:
check = csv_files[0].split('.')
check

In [ ]:
if type(check) is str:
    print('This logic works')

In [ ]:
csv_files[0]

In [ ]:
qct_filepath = generate_file_path(qct_directory, '3305-00115-00128')

In [ ]:
data = {}
with open(qct_filepath, errors='ignore') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.update({reader.line_num:row})

In [ ]:
for key,info in data.items():
    if 'serial number' in [x.lower() for x in info]:
        print(key)
        print(info)

In [ ]:
data

In [ ]:
for key,info in data.items():
    if 'svufoilcoef' in [x.lower() for x in info]:
        print(key)
        print(info)

In [ ]:
info[3:]

In [ ]:
for key,info in data.items():
    if 'conccoef' in [x.lower() for x in info]:
        print(key)
        print(info)

In [ ]:
data